In [ ]:
from data_access import get_recipes_from_db

In [2]:
from pyomo.environ import ConcreteModel, Var, Objective, Constraint, SolverFactory


In [3]:
# pip install fuzzywuzzy
# pip install python-Levenshtein

In [ ]:
import pandas as pd
from pandas import json_normalize
from sqlalchemy import create_engine
import json
from fuzzywuzzy import process
from datetime import datetime
import openai
import json

In [ ]:
# API_KEY=


In [6]:
# Define the file path
food_list_file_path = '/Users/Matt/MealPlanAutomation/food_list.txt'

# Read the contents of the file
with open(food_list_file_path, 'r') as file:
    food_list = file.read()

In [7]:
# Define the file path
file_path = '/Users/Matt/MealPlanAutomation/food_list.txt'

# Read the contents of the file
with open(file_path, 'r') as file:
    lines = file.readlines()

# Initialize variables to store categories and items
data = []
current_category = None

# Parse the contents of the file
for i, line in enumerate(lines):
    line = line.strip()
    if line:  # Check if the line is not empty
        if current_category is None and (i == 0 or not lines[i-1].strip()) and (i + 1 < len(lines) and not lines[i+1].strip()):
            current_category = line
            data.append({'Category': line, 'Item': None})
        else:  # Otherwise, it's an item
            data.append({'Category': None, 'Item': line})
    else:
        current_category = None  # Reset current category on blank line

# Create a DataFrame from the parsed data
df = pd.DataFrame(data)

# Forward fill the 'Category' column to fill None values with the last valid category
df['Category'] = df['Category'].ffill()

food_list = df.dropna(subset=['Item'])


In [ ]:
user = 'root'
password = 'root'
host = 'localhost' #'pgdatabase-1'  # find by running in git bash: docker inspect -f '{{range .NetworkSettings.Networks}}{{.IPAddress}}{{end}}' meal_planning-pgdatabase-1
port = '5432'
db = 'meal_planning'
table_name = 'recipes'
# Create the connection string
connection_string = f'postgresql://{user}:{password}@{host}:{port}/{db}'

# Function to get recipes from the database
def get_recipes_from_db(connection_string, table_name):
    engine = create_engine(connection_string)
    query = f'SELECT * FROM {table_name}'
    recipes_df = pd.read_sql(query, engine)
    return recipes_df

# Try to fetch the data and handle any exceptions
try:
    recipes_df = get_recipes_from_db(connection_string, table_name)
except Exception as e:
    print(f"Error: {e}")

In [10]:
# Split the Categories column on commas
recipes_df['Categories'] = recipes_df['Categories'].str.split(',')

recipes_df

,Title,Ingredients,Categories,Rating,Servings,Difficulty,LastModifiedDate
0,Nothing Fishy Sushi Wraps,"{""60 grams raw cashews, soaked in"",""240 ml boi...","[Dinner, Fibre Fuelled]",0,"4, with leftovers for Nothing Fishy Sushi Bowls",,2025-01-11 18:15:18
1,The Green Drink,"{""360 grams chopped honeydew melon"",""4 kiwis, ...","[Drinks, Fibre Fuelled]",0,2,,2025-01-26 14:58:42
2,Tuscan Kale Soup,"{1.,""5 tablespoons olive oil or garlic-infused...",[],3,4,,2025-01-26 14:58:42
3,Pesto Pasta,"{""Rocket-Walnut Pesto"",""60 grams rocket"",""60 g...","[Dinner, Fibre Fuelled]",4,4,Medium,2025-01-11 18:20:24
4,Creamy Coriander Salad,"{""1 jalapeño, roughly chopped"",""Half a bunch c...","[Fibre Fuelled, Side Salad]",4,4,Medium,2025-01-11 18:15:18
...,...,...,...,...,...,...,...
151,Big Green Noodles,"{""4 tbsp Thai Green Curry Paste"",""400 ml Cocon...",[Mob],0,Serves 4,,2025-01-11 18:15:18
152,Lettuce Cups With Curried Tempeh And Chickpeas,"{""8 ounces/225g tempeh, steamed for"",""20 minut...",[How Not To Diet],0,4,Easy,2025-01-26 14:57:32
153,Sticky Tamarind Chicken,"{""100 g Tamarind Paste"",""3 tbsp Honey"",""3 tbsp...",[Mob],0,Serves 4,,2025-01-26 14:58:42
154,Harissa Chicken,"{""4 Skinless Chicken Thighs"",""6 tbsp Yoghurt"",...",[Mob],0,Serves 4,,2025-01-11 18:15:18


In [20]:
food_list[food_list['Item'].str.contains('Cucumber')]

,Category,Item
139,Other Vegetables,Cucumber


In [20]:
# # Normalize the Ingredients column
# def normalize_ingredients(ingredients_str):
#     # Remove the curly braces and split by '","'
#     ingredients = ingredients_str.strip('{}').split('","')
#     # Remove any leading/trailing quotes and whitespace, and convert to lowercase
#     return [ingredient.strip().strip('"').lower() for ingredient in ingredients]

# recipes_df['Normalized_Ingredients'] = recipes_df['Ingredients'].apply(normalize_ingredients)

# # Create a mapping from ingredients to categories using fuzzy matching
# def map_ingredients_to_categories(ingredients, food_list_df):
#     categories = []
#     for ingredient in ingredients:
#         # Find the best match in the food list
#         best_match, score = process.extractOne(ingredient, food_list_df['Item'])
#         if score >= 80:  # Use a threshold to ensure a good match
#             category = food_list_df[food_list_df['Item'] == best_match]['Category'].values[0]
#         else:
#             category = 'Unknown'
#         categories.append(category)
#     return categories

# recipes_df['Ingredient_Categories'] = recipes_df['Normalized_Ingredients'].apply(map_ingredients_to_categories, food_list_df=food_list)

# # Display the DataFrame
# print("Recipes DataFrame:")
# print(recipes_df[['Title', 'Normalized_Ingredients', 'Ingredient_Categories']])

In [ ]:
daily_dozen = pd.DataFrame(category_dict['Category'].drop_duplicates()).reset_index(drop=True)
daily_dozen['Daily quantity'] = [3, 1, 3, 1, 2, 2, 1, 1, 1, 3]

In [50]:
daily_dozen

,Category,Daily quantity
0,Beans,3
1,Berries,1
2,Other Fruits,3
3,Cruciferous Vegetables,1
4,Greens,2
5,Other Vegetables,2
6,Flaxseeds or Linseeds,1
7,Nuts and Seeds,1
8,Herbs and Spices,1
9,Whole Grains,3


In [32]:

# Create the instructions
instructions = """You will be provided with information containing the following keys: 'Title', 'Ingredients' and 'Servings'.
                  The 'Ingredients' key contains a list of strings, each representing an ingredient and its quantity (likely in grams/millilitre), and possibly some further prepping instruction in a recipe (which we don't need to use). The 'Servings' key represents the number of servings of the meal that the set of ingredients will produce.
                  You will also be provided with information containing the following keys: 'Category' and 'Item'.
                  The 'Items' represent a food item and the 'Category' represents the food category it belongs to. Use this, along with you expert knowledge of plant foods, as your basis to categorise each plant-based ingredient you find. Please only store ingredients that fit into these ten distinct categories (Beans, Berries, Other Fruits, Cruciferous Vegetables, Greens, Other Vegetables, Flaxseeds or Linseeds, Nuts and Seeds, Herbs and Spices, Whole Grains).
                  Any oils, milks and pastes should not be included. Only fresh herbs and spices should be included, no dried.
                  Moreover, use the quantity of each of the ingredients you identify along with the Servings information to calculate the total amount of that food per serving, use the relevant units (e.g. grams, ml, tbsp). Given that the input data is string and has not been cleaned/converted to numeric, you may encounter servings sizes that e.g. give a range or have other ambiguity. Do your best to calculate the amounts per serving given your understanding and knowledge.
                  """

# Create the output format
output_format = """The output should only be a JSON object in the following format, do not include any additional text or explanation of your logic.
                  {
                      "Title": "Nothing Fishy Sushi Wraps",
                      "Ingredients": [
                          {
                              "Ingredient": "cashews",
                              "Quantity per serving": 15 grams,
                              "Category": "Nuts and Seeds"
                          }
                      ]
                  },
                  {
                      "Title": "The Green Drink",
                      "Ingredients": [
                          {
                              "Ingredient": "honeydew melon",
                              "Quantity per serving": 90 grams
                              "Category": "Other Fruits"
                          },
                          {
                              "Ingredient": "kiwis",
                              "Quantity per serving": 1,
                              "Category": "Other Fruits"
                          }
                      ]
                  } 
                """

In [ ]:
class RecipeProcessor:
    def __init__(self, recipes_df, category_dict, instructions, output_format):
        self.recipes_df = recipes_df
        self.category_dict = category_dict
        self.instructions = instructions
        self.output_format = output_format

    def get_response(self, prompt, temperature=0.0): #, max_tokens=4096):
        # Create a request to the chat completions endpoint
        response = client.chat.completions.create(
            model="gpt-4o",
            # Assign the role and content for the message
            messages=[{"role": "user", "content": prompt}], 
            temperature = temperature)
        return response.choices[0].message.content

    def process_recipe(self, recipe):
        # Combine the dictionaries into a single string
        dictionaries = f"ingredients_dict = {recipe}\ncategory_dict = {self.category_dict}"

        # Create the final prompt
        prompt = self.instructions + self.output_format + f"\n\nHere are the input dictionaries:\n```{dictionaries}```"

        # Get the response from the ChatGPT API
        response = self.get_response(prompt)

        # Remove the Markdown code block delimiters and the 'json' prefix
        if response.startswith('```json'):
            response = response[7:-3].strip()

        # Convert the string to a JSON object
        json_object = json.loads(response)

        return json_object

    def process_all_recipes(self, last_modified_date, num_recipes=None):
        results = []

        # Convert the input last_modified_date to a datetime object
        last_modified_date = datetime.strptime(last_modified_date, '%Y-%m-%d')

        # Determine the number of recipes to process
        if num_recipes is None:
            num_recipes = len(self.recipes_df)

        # Loop through each row in the DataFrame
        for i in range(num_recipes):
            # Extract the row as a dictionary
            recipe = self.recipes_df.iloc[i].to_dict()

            # Filter recipes based on the last modified date
            recipe_last_modified_date = recipe['LastModifiedDate']
            if isinstance(recipe_last_modified_date, pd.Timestamp):
                recipe_last_modified_date = recipe_last_modified_date.to_pydatetime()
            if recipe_last_modified_date > last_modified_date:
                # Process the recipe
                json_object = self.process_recipe(recipe)

                # Append the JSON object to the results list
                results.append(json_object)

        return results

    def flatten_results(self, results):
        # Flatten the JSON structure and create a DataFrame
        df = pd.concat([json_normalize(result, 'Ingredients', ['Title']) for result in results], ignore_index=True)
        return df

In [17]:
ingredients_dict = dict(recipes_df[['Title', 'Ingredients', 'Servings']])
category_dict = dict(food_list)

In [ ]:
# only call the OpenAI API for recipes that have been modified after this dateß
last_modified_date = '2025-01-25' 

# Initialize the RecipeProcessor
processor = RecipeProcessor(recipes_df, category_dict, instructions, output_format)

# Process the recipes
results = processor.process_all_recipes(last_modified_date, num_recipes=10)
df = processor.flatten_results(results)

In [44]:
df

,Ingredient,Quantity per serving,Category,Title
0,honeydew melon,180,Other Fruits,The Green Drink
1,kiwis,2,Other Fruits,The Green Drink
2,mint leaves,5,Herbs and Spices,The Green Drink
3,white onion,0.125 small,Other Vegetables,Tuscan Kale Soup
4,celery,0.5 stick,Other Vegetables,Tuscan Kale Soup
5,carrot,0.25,Other Vegetables,Tuscan Kale Soup
6,diced tomatoes,100 grams,Other Fruits,Tuscan Kale Soup
7,quinoa,22.5 grams,Whole Grains,Tuscan Kale Soup
8,kale,0.5 large handful,Greens,Tuscan Kale Soup
9,white beans,100 grams,Beans,Tuscan Kale Soup


In [ ]:
# environment:
    # - DB_USER=root
    # - DB_PASSWORD=root
    # - DB_HOST=pgdatabase
    # - DB_PORT=5432
    # - DB_NAME=meal_planning
    # - TABLE_NAME=recipes
    # - REPO_URL=https://github.com/Matt1303/recipe_html_pages
    # - CLONE_DIR=/app/clone_dir

In [60]:
### NEXT: upload the data to the postgres database in a new table and allow this table to be overwritten for any new recipes or changes to the existing recipes

# def main():
    # user = os.getenv('DB_USER')
    # password = os.getenv('DB_PASSWORD')
    # host = os.getenv('DB_HOST')
    # port = os.getenv('DB_PORT')
    # db = os.getenv('DB_NAME')
    # table_name = os.getenv('TABLE_NAME')

def write_to_db(user='root', 
                password='root', 
                host='localhost', #'pgdatabase', 
                port=5432, 
                db='meal_planning', 
                table_name='ingredients_by_recipe',
                ingredients_df=df):
    # Create a connection to the PostgreSQL database
    engine = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{db}')

    # Load the DataFrame into the database, replacing the table if it already exists
    ingredients_df.to_sql(name=table_name, con=engine, if_exists='replace', index=False)
    print(f"Table '{table_name}' created successfully in database '{db}'")

In [61]:
write_to_db()

Table 'ingredients_by_recipe' created successfully in database 'meal_planning'
